## Imports and Setup

In [1]:
import os
import pprint
os.chdir("..")

from src.dfcv_colocation_mapping import data_download
from src.dfcv_colocation_mapping import data_utils
from src.dfcv_colocation_mapping import map_utils

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

%load_ext autoreload
%autoreload 2

## Parameters

In [11]:
iso_code = "NPL"
adm_level = "ADM3"

## Download Datasets

In [12]:
dm = data_download.DatasetManager(
    iso_code, 
    adm_level=adm_level,
    data_dir="data/",
    config_file="src/dfcv_colocation_mapping/configs/data_config.yaml",
    acled_file="creds/acled_creds.yaml",
    dtm_file="creds/dtm_creds.yaml",
    adm_config_file="src/dfcv_colocation_mapping/configs/adm_config.yaml",
    osm_config_file="src/dfcv_colocation_mapping/configs/osm_config.yaml",
    mhs_aggregation="power_mean"
)

In [13]:
# Remove Strategic Developments
# --------- COPY THIS CODE ------------ #
dm.set_acled_filters(
    category="disorder_type", 
    values = [
        'Strategic developments'
    ],
    operation="exclude"
)
# ----------------------------------- #

# Remove peaceful protests
dm.set_acled_filters(
    category="sub_event_type", 
    values = [
        'Peaceful protest'
    ],
    operation="exclude"
)

# Sense-check inclusion/exclusion criteria
pprint.pprint(dm.acled_filters)

{'exclude': {'disorder_type': ['Strategic developments'],
             'event_type': ['Strategic developments'],
             'sub_event_type': ['Agreement',
                                'Arrests',
                                'Change to group/activity',
                                'Disrupted weapons use',
                                'Headquarters or base established',
                                'Looting/property destruction',
                                'Non-violent transfer of territory',
                                'Other',
                                'Peaceful protest']},
 'include': {'disorder_type': ['Political violence',
                               'Political violence; Demonstrations',
                               'Demonstrations'],
             'event_type': ['Protests',
                            'Violence against civilians',
                            'Battles',
                            'Riots',
                            'Explosions/

In [ ]:
dm.data.query("")

In [ ]:
dm.data.query("disorder_type == 'Strategic developments'")

In [ ]:
dm.download_data()
geoplot = map_utils.GeoPlot(dm, data_dir="data/", map_config_file="src/dfcv_colocation_mapping/configs/map_config.yaml")
dm.data.sort_values("mhs_all_ucdp_worldpop_exposure_relative", ascending=False).head(3)

INFO:root:Loading ADM3 geoboundaries...
INFO:root:Downloading geoboundary for NPL...
INFO:root:Geoboundary file saved to C:\Users\Isabelle\Documents\GFDRR\dfcv\colocation-mapping\data/NPL\NPL_ADM1.geojson.
INFO:pyogrio._io:Created 7 records
INFO:root:Geoboundary file saved to C:\Users\Isabelle\Documents\GFDRR\dfcv\colocation-mapping\data/NPL\NPL_ADM2.geojson.
INFO:pyogrio._io:Created 775 records


## AHP Weight Calculation [Optional]

In [ ]:
dm.calculate_ahp()
geoplot = map_utils.GeoPlot(dm)
dm.data.sort_values(f"mhs_all_wbg_acled_{dm.asset}_exposure_relative", ascending=False).head(3)

## Plot Regional Groupings

In [ ]:
ax, x = geoplot.plot_geoboundaries(
    adm_level="ADM2",
    group="ADM1"
);

## Plot Population

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="bem",
    kwargs={"legend_type": "colorbar", "legend_y": 0.475}
)

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="acled_conflict_count",
    kwargs={
        "legend_type": "colorbar",
        "cmap": "Reds",
        "edgecolor": "dimgray",
        "legend_y": 0.475
    }
);

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="numPresentIdpInd",
    data=dm.dtm
)

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var="acled_conflict_count",
    kwargs={
        "legend_type": "colorbar",
        "cmap": "Reds",
        "edgecolor": "dimgray",
        #"legend_y": 0.475
    },
    #zoom_to={"ADM1": "Kigali City"},
);
geoplot.plot_points(
    "event_type", 
    dataset="acled", 
    clustering=True,  
    kwargs={
        "alpha": 0.8,
        "legend_y": 0.435,
        "legend_x": 0.275,
        "markerscale": 2,
        "cmap": "tab10"
    },
    #zoom_to={"ADM1": "Kigali City"},
    ax=ax, xpos=x,
);

## Plot Conflict Exposure - ACLED (Population Best)

In [ ]:
source = "wbg_acled"
geoplot.plot_choropleth( 
    var=f"{source}_bem_exposure",
    subtitle="By township, from 2021 to 2025",
    kwargs={"legend_type": "colorbar"}
)
geoplot.plot_choropleth( 
    var=f"{source}_bem_exposure_relative",
    subtitle="By township, from 2021 to 2025",
    kwargs={"legend_type": "colorbar"}
);

## Plot Conflict Exposure - ACLED (Custom Calculation)

In [ ]:
source = "wbg_acled"
geoplot.plot_choropleth( 
    var=f"{source}_bem_exposure",
)
ax, x = geoplot.plot_choropleth( 
    var=f"{source}_bem_exposure",
    var_bounds=[0, 100],
);

## Plot Conflict Exposure - UCDP

In [ ]:
source = "ucdp"
geoplot.plot_choropleth( 
    var=f"{source}_bem_exposure"
)
geoplot.plot_choropleth( 
    var=f"{source}_bem_exposure_relative",
    var_bounds=[0, 100]
);

## Zoom Choropleth to a specific region

In [ ]:
source = "wbg_acled"
geoplot.plot_choropleth( 
    var=f"{source}_{dm.asset}_exposure",
    zoom_to={"ADM1": "Lomami"},
    subtitle="By township, from 2021 to 2025"
)

In [ ]:
ax = geoplot.plot_choropleth( 
    var=f"{source}_{dm.asset}_exposure_relative",
    zoom_to={"ADM1": "Lakes"},
    subtitle="By township, from 2021 to 2025",
    var_bounds=[0, 100],
    kwargs={
        'legend_shrink': 0.20,
        'legend_type': 'colorbar',
    },
);

In [ ]:
ax = geoplot.plot_choropleth( 
    var=f"numPresentIdpInd",
    subtitle="By township, from 2021 to 2025",
    #var_bounds=[0, 100],
    kwargs={
        'legend_shrink': 0.20,
        'legend_type': 'colorbar',
    },
);

In [ ]:
source = "wbg_acled"
geoplot.plot_choropleth( 
    var=f"{source}_{dm.asset}_exposure",
    zoom_to={"ADM1": "Eastern Province"},
    subtitle="By township, from 2021 to 2025"
)
ax = geoplot.plot_choropleth( 
    var=f"{source}_{dm.asset}_exposure_relative",
    zoom_to={"ADM1": "Eastern Province"},
    subtitle="By township, from 2021 to 2025",
    var_bounds=[0, 100],
);

## Map Exposure to a Single Hazard

In [ ]:
hazard = "earthquake"
geoplot.plot_raster(hazard);

In [ ]:
geoplot.plot_choropleth(f"{hazard}");

In [ ]:
geoplot.plot_choropleth(f"{hazard}_bem_intensity_weighted_exposure_relative");

In [ ]:
geoplot.plot_choropleth(
    f"{hazard}_bem_exposure",
    kwargs={"legend_type": "colorbar"}
);

In [ ]:
geoplot.plot_choropleth(
    f"{hazard}_worldpop_intensity_weighted_exposure_relative",
);

## Plot Multi-hazard Exposure Index and Multi-hazard Conflict Exposure Index

In [ ]:
geoplot.plot_choropleth( 
    var=f"mhs_all_bem_exposure_relative",
    kwargs={
        "legend_type": "colorbar", 
        "linewidth": 0.1
    }
);

In [ ]:
ax, x = geoplot.plot_choropleth( 
    var=f"mhs_all_worldpop_exposure_relative",
    kwargs={"legend_type": "colorbar", "legend_y": 0.5,}
);
geoplot.plot_points(
    "type_of_violence", 
    dataset="acled", 
    clustering=False,  
    kwargs={
        "alpha": 0.5,
        "legend_y": 0.425,
        "markerscale": 0.5,
        "cmap": "tab10"
    },
    ax=ax, xpos=x,
);

In [ ]:
geoplot.plot_choropleth( 
    var=f"mhs_all_wbg_acled_{dm.asset}_intensity_weighted_exposure_relative",
    subtitle="By township, from 2021 to 2025",
    kwargs={"legend_type": "colorbar"}
);

In [ ]:
geoplot.plot_choropleth( 
    var=f"mhs_all_wbg_acled_{dm.asset}_exposure_relative",
    kwargs={"legend_type": "colorbar"}
);

# Bivariate Choropleth: Overlay Multi-hazard and Conflict Exposure

In [ ]:
geoplot.plot_bivariate_choropleth( 
    var1=f"wbg_acled_{dm.asset}_exposure_relative",
    var2=f"mhs_all_{dm.asset}_exposure_relative",
    var1_bounds=[0, 1],
    var2_bounds=[0, 1],
    binning="equal_intervals",
    kwargs={
        "legend_fontsize": 4, 
        "edgecolor": "white", 
        "linewidth": 0.2
    }
);


In [ ]:
geoplot.plot_bivariate_choropleth( 
    var2=f"numPresentIdpInd",
    var1=f"mhs_all_{dm.asset}_intensity_weighted_exposure_relative",
    #var1_bounds=[0, 1],
    #var2_bounds=[0, 1],
    binning="equal_intervals",
    kwargs={
        "legend_fontsize": 4, 
        "edgecolor": "white", 
        "linewidth": 0.2
    }
);

In [ ]:
geoplot.plot_bivariate_choropleth( 
    var1=f"wbg_acled_{dm.asset}_exposure_relative",
    var2=f"mhs_all_{dm.asset}_intensity_weighted_exposure_relative",
    var1_bounds=[0, 1],
    var2_bounds=[0, 1],
    binning="equal_intervals",
    kwargs={
        "legend_fontsize": 4, 
        "edgecolor": "white", 
        "linewidth": 0.2
    }
);

In [ ]:
geoplot.plot_bivariate_choropleth( 
    var1=f"ucdp_{dm.asset}_exposure_relative",
    var2=f"mhs_{dm.asset}_intensity_weighted_exposure_relative",
    #var1_bounds=[0, 1],
    var2_bounds=[0, 1],
    binning="equal_intervals",
    kwargs={
        "legend_fontsize": 4, 
        "edgecolor": "white", 
        "linewidth": 0.2
    }
);

## Zoom Bivariate Choropleth to a specific region

In [ ]:
geoplot.plot_bivariate_choropleth( 
    var1=f"wbg_acled_{dm.asset}_exposure_relative",
    var2=f"mhs_{dm.asset}_intensity_weighted_exposure_relative",
    var1_bounds=[0, 1],
    var2_bounds=[0, 1],
    binning="equal_intervals",
    kwargs={"legend_fontsize": 4},
    zoom_to={"ADM1": "Kigali City"},
);

## Interactive Map

In [ ]:
m = geoplot.plot_folium(
    adm_level=adm_level,
    var="mhs_all_wbg_acled_worldpop_exposure_relative",
    kwargs={"zoom_start": 9}
)
m

In [ ]:
 m.save("folium.html")